In [2]:
from langchain_core.tools import tool

@tool
def add(a:int, b:int) -> int:
    """Adds a and b.
    
    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.
    
    Args:
        a: first int
        b: second int
    """
    return a * b

tools = [add, multiply]

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

class add(BaseModel):
    """Add two integers together."""
    
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")
    
class multiply(BaseModel):
    """Multiply two integers together."""
    
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")
    
tools = [add, multiply]

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")

In [10]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_WXB47RjsAUUKW9ftVQZ4fbSU'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_sovvymChbUIhxFZ9NbminVCa'}]

In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)

In [11]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

In [12]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_YFlLzh3QKbw1d9vfERW2C5dq', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'add', 'args': '', 'id': 'call_VyWlUxU4HXrUHVZxvGmqGeVF', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[]


In [13]:
from langchain_core.messages import HumanMessage, ToolMessage

tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))


AttributeError: type object 'multiply' has no attribute 'invoke'

좀 빡센데?

- tool 만들고
- llm에 binding 해서 사용할 수 있는듯 보임.